# Optimizing "Serial" Performance

At the heart of fast parallel code must be fast serial code. Parallelism can make a good serial code faster. But it can also make a bad code even worse. One can write terribly slow code in any language, including Julia. In this notebook we want to understand what makes Julia code slow and how to detect and avoid common pitfalls. This will lead to multiple concrete performance tips that will help you speed up your Julia code and to write more efficient code in the first place.

By far the most common reasons for slow Julia code are

* **too many (unnecessary) allocations**
* **break-down of type inference** (e.g. type instabilities)

## Avoid unnecessary allocations

Dynamic heap allocations are costly compared to floating point operations. Avoid them, in particular in "hot" loops, because they may trigger garbage collection.

In [ ]:
using BenchmarkTools

In [ ]:
@btime 1.2 + 3.4;
@btime Vector{Float64}(undef, 1);

### Example 1: Element-wise operations

In [ ]:
function f()
  x = [1,2,3]
  for i in 1:100_000
    x = x + 2*x
  end
  return x
end

In [ ]:
@btime f();

* Huge number of allocations!
* Bad sign that they scale with the number of iterations!

#### Fix 1: Write explicit loops

In [ ]:
function f()
    x = [1,2,3]
    for i in 1:100_000
        for k in eachindex(x)
            x[k] = x[k] + 2 * x[k]
        end
    end
    return x
end

@btime f();

#### Fix 2: Broadcasting (aka "More Dots")

(Recommendation: Old but great [blog post](https://julialang.org/blog/2017/01/moredots) by Steven G. Johnson ([related notebook](https://github.com/JuliaLang/www.julialang.org/blob/master/blog/_posts/moredots/More-Dots.ipynb)))

In [ ]:
function f()
    x = [1,2,3]
    for i in 1:100_000
        x = x .+ 2 .* x
    end
    return x
end

@btime f();

In [ ]:
function f()
    x = [1,2,3]
    for i in 1:100_000
        x .= x .+ 2 .* x
        # or put @. in front
    end
    return x
end

@btime f();

#### Fix 3: Immutable datatypes (if possible)

In [ ]:
using StaticArrays

function f()
  x = @SVector [1,2,3]
  for i in 1:100_000
    x = x + 2*x
  end
  return x
end

@btime f();

No dynamic heap allocations at all!

### Example 2: Linear Algebra

In [ ]:
function f()
    A = rand(100,100)
    B = rand(100,100)
    s = 0.0
    for i in 1:1000
        C = A * B
        s += C[i]
    end
    return A
end

In [ ]:
@btime f();

#### Fix: Preallocate and reuse memory + in-place matrix-multipy

In [ ]:
using LinearAlgebra

function f()
    A = rand(100,100)
    B = rand(100,100)
    C = zeros(100,100) # preallocate
    s = 0.0
    for i in 1:1000
        mul!(C, A, B) # reuse / in-place matmul
        s += C[i]
    end
    return A
end

In [ ]:
@btime f();

### Example 3: Array slicing

By default, array-slicing creates copies!

In [ ]:
using BenchmarkTools

X = rand(3,3);

In [ ]:
f(Y) = Y[:,1] .+ Y[:,2] .+ Y[:,3]

@btime f($X);

#### Fix: Views

In [ ]:
f(Y) = @views Y[:,1] .+ Y[:,2] .+ Y[:,3]

# expands to
# f(Y) = view(Y, 1:3, 1) .+ view(Y, 1:3, 2) .+ view(Y, 1:3, 3)

@btime f($X);

(Note that [copying data isn't always bad](https://docs.julialang.org/en/v1/manual/performance-tips/#Copying-data-is-not-always-bad))

### Example 4: Vectorized style

In [ ]:
@btime sum(map(sin, [k for k in 1:10]));

#### Fix: Generators and Laziness

In [ ]:
@btime sum(sin(k) for k in 1:10); # generator

In [ ]:
@btime sum(sin, k for k in 1:10); # two-argument version of sum

In [ ]:
@btime first(map(sin, [k for k in 1:10]));

In [ ]:
@btime first(Iterators.map(sin, [k for k in 1:10])); # lazy map

## Type inference: Avoid type instabilities

**Type stability**: A function `f` is type stable if for a given set of input argument types the return type is always the same.

In particular, it means that the type of the output of `f` cannot vary depending on the **values** of the inputs.

**Type instability**: The return type of a function `f` is not predictable just from the type of the input arguments alone.

Instructive example: `f(x) = rand() > 0.5 ? 1.23 : "string"`

### Example: Global scope

A typical cause of type instability are global variables.

From a compiler perspective, variables defined in global scope **can change their value and even their type(!) any time**.

In [ ]:
a = 2.0
b = 3.0

f() = 2*a+b

In [ ]:
f()

In [ ]:
@code_llvm f()

In [ ]:
@code_warntype f()

#### Fix 1: Work in local scope

In [ ]:
function local_scope()
    a=2.0
    b=3.0
    
    f() = 2a+b
    
    return f() 
end

local_scope()

In [ ]:
@code_llvm local_scope()

This is fast.

In fact, it's not just fast, but **as fast as it can be**! Julia has figured out the result of the calculation at compile-time and returns just the literal, i.e. `local_scope() = 7`.

In [ ]:
@code_warntype local_scope()

#### Fix 2: Make globals `const`ant

In [ ]:
const A=2.0
const B=3.0

f() = 2A+B

f()

In [ ]:
@code_llvm f()

In [ ]:
@code_warntype f()

#### Fix 3: Write self-contained functions

In [ ]:
f(a,b) = 2a+b

In [ ]:
@code_llvm debuginfo=:none f(2.0,3.0)

**Write functions not scripts!**

### Example: Multiple `return` statements

In [ ]:
function f(x, flag)
    if flag
        return 1:3
    else
        return [1,2,3]
    end
end

In [ ]:
@code_warntype f(rand(10), true)

In [ ]:
typeof(f(x, true))

In [ ]:
typeof(f(x, false))

#### Fix: Single `return` statement

In [ ]:
function f(x, flag)
    result = Vector{Int64}(undef, 3)
    if flag
        result .= 1:3
    else
        result .= [1,2,3]
    end
    return result
end

In [ ]:
@code_warntype f(x, true)

## Type inference: Avoid abstract field types

A common reason for type inference to break are not-concretely typed fields in `struct`s.

### Example

In [ ]:
using BenchmarkTools

In [ ]:
struct MyType
    x::Number
    y
end

f(a::MyType) = a.x^2 + sqrt(a.x)

In [ ]:
a = MyType(3.0, "test")

@code_warntype f(a);

In [ ]:
@btime f($a);

In [ ]:
typeof(a)

**Note:** Technically not a type instability according to our definition because the return type is always `Any`.

**"Type stability"**: A function `f` is type stable if for a given set of input argument types the return type is always the same and *concrete*.

#### Fix 1: Concrete typing

In [ ]:
struct MyTypeConcrete
    x::Float64
    y::String
end

f(b::MyTypeConcrete) = b.x^2 + sqrt(b.x)

In [ ]:
b = MyTypeConcrete(3.0, "test")
@code_warntype f(b)

In [ ]:
@btime f($b);

#### Fix 2: Type parameters

But what if I want to accept any kind of, say, `Number` and `AbstractString` for our type?

In [ ]:
struct MyTypeParametric{A<:Number, B<:AbstractString}
    x::A
    y::B
end

f(c::MyTypeParametric) = c.x^2 + sqrt(c.x)

In [ ]:
c = MyTypeParametric(3.0, "test")

In [ ]:
@code_warntype f(c)

From the type alone the compiler knows what the structure contains and can produce optimal code:

In [ ]:
@btime f($c);

In [ ]:
c = MyTypeParametric(Float32(3.0), SubString("test"))

In [ ]:
@btime f($c);

## Type inference: Avoid untyped containers

### Example

In [ ]:
function f()
    numbers = []
    for i in 1:10
        push!(numbers, i)
    end
    sum(numbers)
end

@btime f();

In [ ]:
@code_warntype f()

In [ ]:
typeof([])

In [ ]:
function f()
    numbers = Int[]
    for i in 1:10
        push!(numbers, i)
    end
    sum(numbers)
end

@btime f();

In [ ]:
@code_warntype f()

## Type inference: Avoid changing variable types

Variables in a function should not change type.

### Example

In [ ]:
function f()
    x = 1
    for i = 1:10
        x /= rand()
    end
    return x
end

In [ ]:
@code_warntype f()

(On a side note: since the type can only vary between `Float64` and `Int64`, Julia can still produce reasonable code by *union splitting*. I recommend reading [this blog post](https://julialang.org/blog/2018/08/union-splitting) by Tim Holy.)

#### Fix 1: Initialize with correct type

In [ ]:
function f()
    x = 1.0
    for i = 1:10
        x /= rand()
    end
    return x
end

In [ ]:
@code_warntype f()

In [ ]:
@code_warntype h()

#### Fix 2: Specify types (to get errors or to heal the problem by conversion)

In [ ]:
function f()
    x::Float64 = 1 # implicit conversion
    for i = 1:10
        x /= rand()
    end
    return x
end

In [ ]:
@code_warntype f()

#### Fix 3: Special-case first iteration

In [ ]:
function f()
    x = 1/rand()
    for i = 2:10
        x /= rand()
    end
    return x
end

In [ ]:
@code_warntype f()

## Type inference: Isolate unavoidable type instabilities

Type instabilities can occur very naturally, for example when reading unknown user files or user input. Hence, not every instability can be avoided.

If that's the case, isolate your expensive computation from the instability by putting it in a separate *kernel function* (also known as introducing a *function barrier*).

In [ ]:
data = Union{Int64,Float64,String}[4, 2.0, "test", 3.2, 1]

In [ ]:
function computation(data)
    x = 1.0
    for i in 1:100
        x = sin(data[1])
        x += data[2]
        x *= data[4]
    end
    return x
end

In [ ]:
@code_warntype computation(data)

In [ ]:
@btime computation($data);

In [ ]:
function computation(data)
    a = data[1]
    b = data[2]
    c = data[4]
    return _computation_kernel(a,b,c)
end

function _computation_kernel(a,b,c)
    x = 1.0
    for i in 1:100
        x = sin(a)
        x += b
        x *= c
    end
    return x
end

In [ ]:
@code_warntype computation(data)

In [ ]:
@code_warntype _computation_kernel(data[1], data[2], data[4])

In [ ]:
@btime computation($data);

Note that the computational kernel function is fully type inferred.

## General performance tips

### Access arrays in column-major order

<img src="../imgs/column-major-2D.png" width=800px>
(<a href=https://mitmath.github.io/18337/lecture2/optimizing>Image source</a>)

**Fastest varying loop index goes first.**

In [ ]:
M = rand(1000,1000);

function fcol(M)
    for col in 1:size(M, 2)
        for row in 1:size(M, 1)
            M[row, col] = 42
        end
    end
    nothing
end

function frow(M)
    for row in 1:size(M, 1)
        for col in 1:size(M, 2)
            M[row, col] = 42
        end
    end
    nothing
end

In [ ]:
@btime fcol($M)

In [ ]:
@btime frow($M)

Lots of cache misses for `frow`!

### Performance annotations

#### `@inbounds`

Disables bounds checks. (Julia may segfault if you use it wrongly!)

In [ ]:
function f()
    x = [1,2,3]
    for i in 1:100_000
        for k in 1:3
            x[k] = x[k] + 2 * x[k]
        end
    end
    return x
end

@btime f();

In [ ]:
function f_inbounds()
    x = [1,2,3]
    for i in 1:100_000
        for k in 1:3
            @inbounds x[k] = x[k] + 2 * x[k]
        end
    end
    return x
end

@btime f_inbounds();

#### `@simd`

Enables SIMD optimizations that are potentially *unsafe*. Julia may execute loop iterations in arbitrary or overlapping order.

In [ ]:
function f(x)
    s = zero(eltype(x))
    for xi in eachindex(x)
        s += xi
    end
    return s
end

In [ ]:
x = rand(1000);

In [ ]:
@btime f($x);

In [ ]:
function f_simd(x)
    s = zero(eltype(x))
    @simd for xi in eachindex(x)
        s += xi
    end
    return s
end

In [ ]:
@btime f_simd($x);

(For integer input both versions have the same speed because integer addition is associative, in contrast to floating point arithmetics.)

#### `@fastmath`

Enables lots of floating point optimizations that are potentially *unsafe*! It trades accuracy for speed, so, [Beware of fast-math](https://simonbyrne.github.io/notes/fastmath/). (See the [LLVM Language Reference Manual](https://llvm.org/docs/LangRef.html#fast-math-flags) for more information on which compiler options it sets.)

There is `julia --math-mode=fast` to enable fast math globally.

##### Harmless example: FMA - Fused Multiply Add

In [ ]:
f(a,b,c) = a*b+c

In [ ]:
@code_native debuginfo=:none f(1.0,2.0,3.0)

<img src="../imgs/skylake_microarchitecture.png" width=700px>

**Source:** [Intel® 64 and IA-32 Architectures Optimization Reference Manual](https://software.intel.com/sites/default/files/managed/9e/bc/64-ia-32-architectures-optimization-manual.pdf)

In [ ]:
f_fastmath(a,b,c) = @fastmath a*b+c

In [ ]:
@code_native debuginfo=:none f_fastmath(1.0,2.0,3.0)

(In this specific case, [MuladdMacro.jl](https://github.com/SciML/MuladdMacro.jl) is a *safe* alternative.)

### CPU operations vary in cost

http://ithare.com/infographics-operation-costs-in-cpu-clock-cycles/

#### Example: Division vs multiplication

In [ ]:
x = rand(1000)
@btime $x ./ 1000;
@btime $x .* 1e-3;

## Analysis Tools

### [Traceur.jl](https://github.com/MikeInnes/Traceur.jl)

**Basic automatic performance trap checker**. Essentially a codified version of the [performance tips](https://docs.julialang.org/en/v1/manual/performance-tips/) in the Julia documentation.

Important macro: [`@trace`](http://traceur.junolab.org/latest/#Traceur.@trace)

In [ ]:
using Traceur

a = 2.0
b = 3.0

f() = 2*a+b

@trace f()

### [JET.jl](https://github.com/aviatesk/JET.jl)

**Static** code analyzer. (Doesn't execute the code!)

Important macros:
* `@report_opt`: check for potential optimization problems ([optimization analysis](https://aviatesk.github.io/JET.jl/stable/optanalysis/))
* `@report_call`: check for potential (general) errors ([error analysis](https://aviatesk.github.io/JET.jl/stable/jetanalysis/))

In [ ]:
using JET

a = 2.0
b = 3.0

f() = 2*a+b

@report_opt f() # check for possible optimization problems

In [ ]:
f() = x + 2

@report_call f() # check for possible errors

In [ ]:
@report_opt f()

In [ ]:
@report_call sum("Stuttgart")

### [Cthulhu.jl](https://github.com/JuliaDebug/Cthulhu.jl)

**Interactive code explorer** that let's you navigate through a nested function call-tree and apply macros like `@code_*`, or `@which`, and more. For example, one can recursively apply `@code_warntype` at different levels to detect the origin of a type instability. (Note though that it might take some time to master Cthulhu.)

Important macro: `@descend` (or directly `@descend_code_warntype`)

(Cthulhu isn't a debugger! It has only "static" information.)

In [ ]:
using Cthulhu

A = rand(10,10)
B = rand(10,10)

# @descend A*B # doesn't work in Jupyter -> use REPL

# Core messages of this Notebook

* **Wrap code in self-contained functions** in performance critical applications, i.e. avoid global scope.
* Write **type-stable code** (check with `@code_warntype`).
* Use **views** instead of copies to avoid unnecessary allocations.
* Use **broadcasting (more dots)** to avoid temporary allocations in vectorized code (or write out loops).
* **Types should always have concrete fields.** If you don't know them in advance, use type parameters.
* Be aware of **column major order** when looping over arrays.